In [1]:
import os
import re
import glob
import time
import argparse
import numpy as np
import pandas as pd
import nibabel as nib
from medpy.metric.binary import hd, dc, jc, assd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

In [2]:
info = pd.read_csv(os.path.join("data/MMs", 'volume_info.csv'))
info

,External code,VendorName,Vendor,Centre,ED,ES,Partition,Labeled
0,A0S9V9,Siemens,A,1,0,9,Training,True
1,A1D0Q7,Philips,B,2,0,9,Training,True
2,A1D9Z7,Siemens,A,1,22,11,Training,True
3,A1E9Q1,Siemens,A,1,0,9,Training,True
4,A1K2P5,Canon,D,5,33,11,Testing,False
...,...,...,...,...,...,...,...,...
340,T2Z1Z9,Canon,D,5,29,9,Testing,False
341,T9U9W2,Siemens,A,1,0,10,Training,True
342,V4W8Z5,GE,C,4,19,9,Testing,False
343,W5Z4Z8,Philips,B,2,29,11,Training,True


In [3]:
def load_nii(img_path):
    """
    Function to load a 'nii' or 'nii.gz' file, The function returns
    everyting needed to save another 'nii' or 'nii.gz'
    in the same dimensional space, i.e. the affine matrix and the header

    Parameters
    ----------

    img_path: string
    String with the path of the 'nii' or 'nii.gz' image file name.

    Returns
    -------
    Three element, the first is a numpy array of the image values,
    the second is the affine transformation of the image, and the
    last one is the header of the image.
    """
    nimg = nib.load(img_path)
    return nimg.get_fdata().squeeze(), nimg.affine, nimg.header

def save_nii(img_path, data, affine, header):
    """
    Function to save a 'nii' or 'nii.gz' file.

    Parameters
    ----------

    img_path: string
    Path to save the image should be ending with '.nii' or '.nii.gz'.

    data: np.array
    Numpy array of the image data.

    affine: list of list or np.array
    The affine transformation to save with the image.

    header: nib.Nifti1Header
    The header that define everything about the data
    (pleasecheck nibabel documentation).
    """
    nimg = nib.Nifti1Image(data, affine=affine, header=header)
    nimg.to_filename(img_path)

In [4]:
def conv_int(i):
    return int(i) if i.isdigit() else i

def natural_order(sord):
    """
    Sort a (list,tuple) of strings into natural order.

    Ex:

    ['1','10','2'] -> ['1','2','10']

    ['abc1def','ab10d','b2c','ab1d'] -> ['ab1d','ab10d', 'abc1def', 'b2c']

    """
    if isinstance(sord, tuple):
        sord = sord[0]
    return [conv_int(c) for c in re.split(r'(\d+)', sord)]

In [5]:
dir_gt = "data/MMs/Testing/"
lst_gt = sorted(glob.glob(os.path.join(dir_gt, '**', '*sa_gt.nii.gz'), recursive=True), key=natural_order)
print(len(lst_gt))

136


In [6]:
for vol_path in tqdm(lst_gt):
    data, affine, header = load_nii(vol_path)
    external_code = vol_path.split("/")[-2]
    df_entry = info.loc[info["External code"]==external_code]
    ed_volume = data[..., df_entry["ED"]].squeeze()
    es_volume = data[..., df_entry["ES"]].squeeze()
    
    save_nii(vol_path.replace("sa_gt", "sa_ED_gt"), ed_volume, affine, header)
    save_nii(vol_path.replace("sa_gt", "sa_ES_gt"), es_volume, affine, header)